<a href="https://colab.research.google.com/github/YasserJxxxx/RNN_Traductor/blob/main/Traductor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from google.colab import drive
import kagglehub
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from sklearn.model_selection import train_test_split
import shutil
import sys

# --- 1. CONFIGURACIÓN ---
BATCH_SIZE = 64
LATENT_DIM = 256
MAX_SAMPLES = 10000
EPOCHS_PER_INTERACTION = 1
KAGGLE_DATASET = "lonnieqin/englishspanish-translation-dataset"
POSSIBLE_EXTENSIONS = ['.csv', '.txt']

# Variables globales que serán definidas después de la función preprocess_data
input_characters = set()
target_characters = set()
num_encoder_tokens = 0
num_decoder_tokens = 0
max_encoder_seq_length = 0
max_decoder_seq_length = 0
input_token_index = {}
target_token_index = {}
reverse_target_char_index = {}


# --- 2. GESTIÓN Y CARGA DEL DATASET DESDE KAGGLE/DRIVE ---

def load_data_from_kaggle_or_drive(dataset_name):
    """Monta Drive, descarga el dataset si no existe, y devuelve la ruta del archivo de datos."""

    print("Montando Google Drive...")
    # Permite montar el Drive aunque ya esté montado
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive montado con éxito.")

    # Definir rutas
    DRIVE_BASE_PATH = '/content/drive/MyDrive/KaggleDatasets'
    DATASET_FOLDER = dataset_name.split('/')[-1]
    FINAL_PATH = os.path.join(DRIVE_BASE_PATH, DATASET_FOLDER)

    # Crear carpeta si no existe
    os.makedirs(FINAL_PATH, exist_ok=True)

    # Buscar el archivo de datos precargado en Drive
    for root, dirs, files in os.walk(FINAL_PATH):
        for name in files:
            if any(name.endswith(ext) for ext in POSSIBLE_EXTENSIONS):
                full_path_in_drive = os.path.join(root, name)
                print(f"\n✅ Dataset encontrado y precargado en Drive: {full_path_in_drive}")
                return full_path_in_drive

    # 3. Descargar si el archivo NO existe en Drive
    print(f"\nArchivo no encontrado en Drive. Descargando '{dataset_name}' a la caché de Colab...")
    try:
        local_cache_path = kagglehub.dataset_download(dataset_name)

        found_file = None
        for root, dirs, files in os.walk(local_cache_path):
            for name in files:
                if any(name.endswith(ext) for ext in POSSIBLE_EXTENSIONS):
                    found_file = os.path.join(root, name)
                    break
            if found_file:
                break

        if found_file:
            file_name = os.path.basename(found_file)
            full_path_in_drive = os.path.join(FINAL_PATH, file_name)

            shutil.move(found_file, full_path_in_drive)
            print(f"✅ Descarga completada y movida a Google Drive para persistencia.")
            return full_path_in_drive
        else:
             print(f"❌ ERROR: No se encontró ningún archivo .csv o .txt en el directorio descargado.")
             return None

    except Exception as e:
        print(f"\n❌ ERROR: Falló la descarga o el movimiento del dataset. Detalle: {e}")
        return None

# --- EJECUTAR CARGA INICIAL ---
data_path = load_data_from_kaggle_or_drive(KAGGLE_DATASET)

if not data_path:
    print("\nPrograma terminado debido a la falta del dataset.")
    sys.exit()

# --- 3. PREPROCESAMIENTO DE DATOS ---

def preprocess_data(path, max_samples):
    """Carga y tokeniza el dataset, devolviendo todos los mapeos y longitudes."""
    input_texts = []
    target_texts = []
    input_characters = set()
    target_characters = set()

    # Cargar datos usando pandas (lógica de detección de separador)
    try:
        df = None
        for sep in ['|', '\t', ',', ';']:
            try:
                # Intentar el formato más común (3 columnas, usamos 1 y 2)
                df = pd.read_csv(path, sep=sep, encoding='utf-8', header=None, on_bad_lines='skip', usecols=[1, 2])
                break
            except Exception:
                # Intentar el formato básico (2 columnas, usamos 0 y 1)
                try:
                    df = pd.read_csv(path, sep=sep, encoding='utf-8', header=None, on_bad_lines='skip', usecols=[0, 1])
                    break
                except Exception:
                    continue

        if df is None or df.empty:
            raise ValueError("No se pudo leer el archivo con delimitadores estándar.")

        input_texts_raw = df.iloc[:, 0].astype(str).tolist()
        target_texts_raw = df.iloc[:, 1].astype(str).tolist()

    except Exception as e:
        print(f"Error: Falló la lectura del archivo '{path}'. Detalle: {e}")
        sys.exit()

    # Procesamiento del texto
    for input_text, target_text in zip(input_texts_raw[:min(max_samples, len(input_texts_raw))],
                                      target_texts_raw[:min(max_samples, len(target_texts_raw))]):

        input_text = input_text.strip()
        target_text = '\t' + target_text.strip() + '\n' # Token de inicio y fin

        input_texts.append(input_text)
        target_texts.append(target_text)

        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)

    # --- Generar Mapeos y Longitudes ---

    input_characters = sorted(list(input_characters))
    target_characters = sorted(list(target_characters))
    num_encoder_tokens = len(input_characters)
    num_decoder_tokens = len(target_characters)

    max_encoder_seq_length = max([len(txt) for txt in input_texts if txt]) if input_texts else 1
    max_decoder_seq_length = max([len(txt) for txt in target_texts if txt]) if target_texts else 1

    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

    # Devolvemos todos los resultados
    return (input_texts, target_texts,
            input_token_index, target_token_index, reverse_target_char_index,
            num_encoder_tokens, num_decoder_tokens,
            max_encoder_seq_length, max_decoder_seq_length)

# --- LLAMADA A PREPROCESS_DATA Y ASIGNACIÓN DE GLOBALES ---

(input_texts, target_texts,
 input_token_index, target_token_index, reverse_target_char_index,
 num_encoder_tokens, num_decoder_tokens,
 max_encoder_seq_length, max_decoder_seq_length) = preprocess_data(data_path, MAX_SAMPLES)

print(f"\n--- Resumen de Datos ---")
print(f"Total de muestras: {len(input_texts)}")
print(f"Tamaño del vocabulario de entrada (Inglés): {num_encoder_tokens}")
print(f"Tamaño del vocabulario de salida (Español): {num_decoder_tokens}")
print(f"Longitud máxima de secuencia de entrada: {max_encoder_seq_length}")
print(f"Longitud máxima de secuencia de salida: {max_decoder_seq_length}")


def vectorize_data(input_texts_list, target_texts_list):
    """Convierte las secuencias de texto a matrices one-hot."""
    encoder_input_data = np.zeros(
        (len(input_texts_list), max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')
    decoder_input_data = np.zeros(
        (len(input_texts_list), max_decoder_seq_length, num_decoder_tokens),
        dtype='float32')
    decoder_target_data = np.zeros(
        (len(input_texts_list), max_decoder_seq_length, num_decoder_tokens),
        dtype='float32')

    for i, (input_text, target_text) in enumerate(zip(input_texts_list, target_texts_list)):
        for t, char in enumerate(input_text):
            if char in input_token_index:
                encoder_input_data[i, t, input_token_index[char]] = 1.
        for t, char in enumerate(target_text):
            if char in target_token_index:
                decoder_input_data[i, t, target_token_index[char]] = 1.
                if t > 0:
                    decoder_target_data[i, t - 1, target_token_index[char]] = 1.

    return encoder_input_data, decoder_input_data, decoder_target_data

# Dividir y vectorizar para entrenamiento inicial
input_train, _, target_train, _ = train_test_split(
    input_texts, target_texts, test_size=0.1, random_state=42)

encoder_input_data, decoder_input_data, decoder_target_data = vectorize_data(
    input_train, target_train)


# --- 4. MODELO SEQ2SEQ Y FUNCIONES DE INFERENCIA ---

def create_seq2seq_model():
    """Define y compila el modelo Seq2Seq y sus modelos de inferencia."""
    encoder_inputs = Input(shape=(None, num_encoder_tokens))
    encoder = LSTM(LATENT_DIM, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None, num_decoder_tokens))
    decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(LATENT_DIM,))
    decoder_state_input_c = Input(shape=(LATENT_DIM,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return model, encoder_model, decoder_model

# Crear y entrenar inicialmente el modelo
model, encoder_model, decoder_model = create_seq2seq_model()

print("\n--- Entrenamiento Inicial (5 Épocas Rápidas) ---")
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=5,
          verbose=1)

print("\nModelo inicial entrenado y listo para la traducción interactiva.")


def decode_sequence(input_seq):
    """Decodifica la secuencia de entrada (Inglés) a la de salida (Español)."""
    states_value = encoder_model.predict(input_seq, verbose=0)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        if sampled_token_index not in reverse_target_char_index:
             sampled_char = ''
        else:
             sampled_char = reverse_target_char_index[sampled_token_index]

        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            break

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence.strip()


def get_input_seq(text):
    """Convierte el texto del usuario a la secuencia one-hot necesaria."""
    text = text[:max_encoder_seq_length]

    encoder_input_data_user = np.zeros(
        (1, max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')

    for t, char in enumerate(text):
        if char in input_token_index:
            encoder_input_data_user[0, t, input_token_index[char]] = 1.

    return encoder_input_data_user

# --- 5. BUCLE INFINITO DE TRADUCCIÓN Y APRENDIZAJE ---

print("\n--- INICIO DEL BUCLE INTERACTIVO (APRENDIZAJE CONTINUO) ---")
print("Introduce 'salir' o 'quit' para terminar.")

new_input_texts = []
new_target_texts = []

while True:
    try:
        input_sentence = input("\nIntroduce frase u oración (Inglés): ").strip()

        if input_sentence.lower() in ['salir', 'quit']:
            print("Cerrando el bucle interactivo. ¡Adiós!")
            break

        if not input_sentence:
            continue

        input_seq = get_input_seq(input_sentence)
        translated_text = decode_sequence(input_seq)

        print(f"Traducción del modelo: **{translated_text}**")

        is_correct = input("¿Es correcta la traducción (s/n)? ").strip().lower()

        if is_correct == 'n':
            correct_translation = input("Introduce la traducción correcta (Español): ").strip()

            if correct_translation:
                target_text_corrected = '\t' + correct_translation + '\n'

                new_input_texts.append(input_sentence)
                new_target_texts.append(target_text_corrected)

                print("👍 Par de corrección almacenado.")

                if len(new_input_texts) >= BATCH_SIZE:
                    print(f"\n--- Re-entrenando con {len(new_input_texts)} nuevos pares (EPOCHS={EPOCHS_PER_INTERACTION}) ---")

                    new_encoder_data, new_decoder_input, new_decoder_target = vectorize_data(
                        new_input_texts, new_target_texts)

                    model.fit([new_encoder_data, new_decoder_input], new_decoder_target,
                              batch_size=BATCH_SIZE,
                              epochs=EPOCHS_PER_INTERACTION,
                              verbose=0)

                    print("✅ Modelo actualizado. ¡Aprendí de mis errores!")
                    new_input_texts = []
                    new_target_texts = []

        elif is_correct == 's':
            print("¡Excelente! Traducción validada.")

        else:
            print("Respuesta no válida. Saltando al siguiente intento.")

    except KeyboardInterrupt:
        print("\nCerrando el bucle interactivo por interrupción del teclado. ¡Adiós!")
        break
    except Exception as e:
        print(f"\nOcurrió un error inesperado: {e}")
        print("Continuando con el ciclo.")

Montando Google Drive...
Mounted at /content/drive
Google Drive montado con éxito.

✅ Dataset encontrado y precargado en Drive: /content/drive/MyDrive/KaggleDatasets/englishspanish-translation-dataset/data.csv

--- Resumen de Datos ---
Total de muestras: 10000
Tamaño del vocabulario de entrada (Inglés): 71
Tamaño del vocabulario de salida (Español): 86
Longitud máxima de secuencia de entrada: 17
Longitud máxima de secuencia de salida: 42

--- Entrenamiento Inicial (5 Épocas Rápidas) ---
Epoch 1/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 49s 324ms/step - loss: 1.4491
Epoch 2/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 82s 322ms/step - loss: 1.2746
Epoch 3/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 44s 311ms/step - loss: 1.2661
Epoch 4/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 44s 310ms/step - loss: 1.2559
Epoch 5/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 44s 312ms/step - loss: 1.2368

Modelo inicial entrenado y listo para la traducción interactiva.

--- INICIO DEL BUCLE INTERACTIVO (APRENDIZAJE CONTINUO) ---
Introduce 'salir' o 'quit' para term